In [2]:
import pandas as pd
import numpy as np
from decimal import Decimal as Dec
import matplotlib.pyplot as plt
import sympy

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [78]:
folder_path = "numerical_detail"
num_results_df = pd.DataFrame()

for csv in sorted(os.listdir(folder_path)):
    row_dict = {}
    if "epoc" in csv:
        df = pd.read_csv(folder_path + "/" + csv)
        # df = df.iloc[:-1]
        row_dict['pool'] = csv
        row_dict['pnl'] = sum(df['end_wealth'] - df['holding_wealth']) / max(df['invest_wealth'])
        b = float(csv[:-9].replace("_", "."))
        row_dict['fee_wealth'] = sum(df['end_wealth_fee']) / max(df['invest_wealth'])
        row_dict['IL'] = sum(df['end_wealth_in_pool'] - df['holding_wealth']) / max(df['invest_wealth'])
        row_dict['theo_IL'] = -(np.exp(b) - np.exp(-b)) / 4 * 20 * len(df)  / max(df['invest_wealth'])
        row_dict['effective_tx_ratio'] = sum(df['epoc_effective_tx_count']) / df.iloc[-1]['epoc_end_index']
        num_results_df = num_results_df.append(row_dict, ignore_index=True)
demo_df = num_results_df[['pool', 'pnl','fee_wealth', 'IL', 'theo_IL', 'effective_tx_ratio']]
demo_df
demo_df.to_csv("numerical_detail.csv")

In [85]:
source_folders = "pool_detail"
results_df = pd.DataFrame()
for pool in sorted(os.listdir(source_folders)):
    try:
        row_dict = {}
        folder_path = source_folders + "/" + pool
        profit_dict = {}

        row_dict['pool'] = pool
        for csv in sorted(os.listdir(folder_path)):
            if "epoc" in csv and "optimum" not in csv:
                df = pd.read_csv(folder_path + "/" + csv)
                profit_dict[csv] = sum(df['end_wealth'] - df['holding_wealth']) / max(df['invest_wealth'])
        max_pnl = max(profit_dict.values())
        for (k, v) in profit_dict.items():
            if v == max_pnl:
                row_dict['best_b'] = k
                row_dict['best_b_apr'] = v
                row_dict['b'] = float(k[:-9].replace("_", "."))
                # row_dict['theo_IL'] = -(np.exp(row_dict['b']) - np.exp(-row_dict['b'])) / 4 * 200 / max(df['invest_wealth'])
                # row_dict['best_IL'] = sum((df['holding_wealth'] - df['end_wealth']) * (df['epoc_end_index'] - df['epoc_begin_index'])) / df.iloc[-1]['epoc_end_index']
                row_dict['best_fee_apr'] = sum(df['end_wealth_fee']) / max(df['invest_wealth'])
                row_dict['best_IL_apr'] = sum(df['end_wealth_in_pool'] - df['holding_wealth']) / max(df['invest_wealth'])
                break

        csv = "optimum_epoc.csv"
        df = pd.read_csv(folder_path + "/" + csv)
        row_dict['optimum_b'] = csv
        row_dict['optimum_b_apr'] = sum(df['end_wealth'] - df['holding_wealth']) / max(df['invest_wealth'])
        # row_dict['optimum_IL'] = sum((df['holding_wealth'] - df['end_wealth']) * (df['epoc_end_index'] - df['epoc_begin_index'])) / df.iloc[-1]['epoc_end_index']
        row_dict['optimum_fee_apr'] = sum(df['end_wealth_fee']) / max(df['invest_wealth'])
        row_dict['optimum_IL_apr'] = sum(df['end_wealth_in_pool'] - df['holding_wealth']) / max(df['invest_wealth'])
        results_df = results_df.append(row_dict, ignore_index=True)
    except Exception as e: pass

In [86]:
demo_df = results_df[['pool', 'best_b', 'best_b_apr', 'best_fee_apr', 'best_IL_apr', 'optimum_b_apr', 'optimum_fee_apr', 'optimum_IL_apr']]
demo_df.to_csv("pool_results.csv")